## 3차 과제 - 당뇨병을 예측해보자
- 주제: 데이터 분석 실습(머신러닝)
- 데이터: 피마 인디언 당뇨병 데이터
https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database
- 사용 라이브러리: numpy, pandas, matplotlib, sklearn


### 1. EDA, 전처리

1.1 분석을 위해 사용될 라이브러리들을 import 하세요.

numpy, pandas, matplotlib, sklearn(train_test_split, accurarcy_score, precision_score, recall_score, roc_auc_score, f1_score, confusion_matrix, precision_recall_curve, roc_curve, StandardScaler, LogisticRegression)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

1.2 데이터를 불러오고 타겟 변수('Outcome')에 속한 값들의 비율과(value_counts 활용) 전체 데이터 앞 7개 값을 확인하세요.

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

dataset = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/diabetes.csv')
dataset.head(7)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
5,5,116,74,0,0,25.6,0.201,30,0
6,3,78,50,32,88,31.0,0.248,26,1


In [3]:
dataset.Outcome.value_counts(normalize=True)

0    0.651042
1    0.348958
Name: Outcome, dtype: float64

1.3 데이터의 feature 타입과 Null 개수를 확인하세요.

In [4]:
dataset.dtypes

Pregnancies                   int64
Glucose                       int64
BloodPressure                 int64
SkinThickness                 int64
Insulin                       int64
BMI                         float64
DiabetesPedigreeFunction    float64
Age                           int64
Outcome                       int64
dtype: object

In [5]:
dataset.isnull().sum()

# Null값 없음

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

1.4 예측을 위해 데이터를 feature 데이터 셋과 target 데이터 셋으로 나누세요.

In [6]:
X = dataset.iloc[:, :-1].values  # Outcome을 제외한 모든 피처값
y = dataset.iloc[:, -1].values  # Outcome 피처값들

1.5 X와 y로 나눈 데이터 셋을 다시 train, test로 나누세요(test_size = 0.3, random_state=123, stratify=y).

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 123, stratify=y)

1.6 test size는 어느정도로 해야 좋은지 그리고 왜 그래야 하는지 작성하세요.

1.7 startify 값을 설정하는 이유와 언제 주로 사용되는지 서술하세요.

훈련데이터와 테스트 데이터로 나눌 때, 타깃 데이터가 이진 변수라면 0과 1값을 각 세트에 (test_size에서 지정해준) 비율을 유지하여 나누기 위해 사용한다.

### 2. 예측 및 분석

2.1 성능 평가에 사용될 함수를 만들어보겠습니다. accuracy_score, precision_score, recall_score, confusion_matrix, f1_score, roc_auc_score로 구성되어 있으며 이를 한번에 출력할 수 있도록 합니다.

In [26]:
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score

def clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    #accuracy, precision, recall, f1, roc_auc(roc_auc_score의 경우 y_test와 pred_proba로 측정)는 직접 작성

    accuracy = accuracy_score(y_test, pred)  # 정확도. unbalance한 데이터에서는 오류 가능성
    precision = precision_score(y_test, pred)  # 정밀도
    recall = recall_score(y_test, pred)   #
    f1 = f1_score(y_test, pred)  # Fa: recall과 precision의 조화평균. 데이터가 unbalance한 경우에도 정상적으로 작동함

    # y_test를 1차원 배열로 변환
    y_test_flat = y_test.reshape(-1)

    if pred_proba is not None:
        roc_auc = roc_auc_score(y_test_flat, pred_proba)
    else:
        roc_auc = None

    return confusion, accuracy, precision, recall, f1, roc_auc



confusion_matrix, accuracy, precision, recall, f1_score, roc_auc_score = clf_eval(y_test, pred, pred_proba)
print('오차 행렬,\n', confusion)
print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:4f}, F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

ValueError: y should be a 1d array, got an array of shape (231, 2) instead.

2. 위에서 사용한 성능지표인 Confusion Matrix에 대해 설명하세요.

실제 데이터와 예측한 데이터를 비교하여 맞는 예측과 틀린 예측을 확인해 예측 성능을 측정하기 위한 표이다.

```
# TRUE NEGATIVE (TN)     FALSE POSITIVE (FP)
 불합격일거야 (예측)    합격일거야 (예측)
 불합격 (실제)          불합격 (실제)

 FALSE NEGATIVE (FN)    TRUE POSITIVE (TP)
 불합격일거야 (예측)    합격일거야 (예측)
 합격 (실제)            합격 (실제)           

```
위와 같은 순서로 각각의 사례의 개수를 출력한다.




2.2 로지스틱 회귀로 예측 후 위에서 만든 clf_eval() 함수를 통해 잘 예측했는지 확인하세요(solver='linear'로 설정, 예측 변수명은 pred, pred_proba).

In [9]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [10]:
pred = classifier.predict(X_test)
pred

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0])

In [15]:
pred_proba = classifier.predict_proba(X_test)
pred_proba

array([[0.80395128, 0.19604872],
       [0.96506855, 0.03493145],
       [0.94026186, 0.05973814],
       [0.80236496, 0.19763504],
       [0.36490959, 0.63509041],
       [0.8568911 , 0.1431089 ],
       [0.82936403, 0.17063597],
       [0.94926663, 0.05073337],
       [0.86062111, 0.13937889],
       [0.64326054, 0.35673946],
       [0.77763361, 0.22236639],
       [0.85641279, 0.14358721],
       [0.52416924, 0.47583076],
       [0.87971814, 0.12028186],
       [0.1238729 , 0.8761271 ],
       [0.42431695, 0.57568305],
       [0.76272096, 0.23727904],
       [0.87165193, 0.12834807],
       [0.62394183, 0.37605817],
       [0.80932229, 0.19067771],
       [0.31629409, 0.68370591],
       [0.70662866, 0.29337134],
       [0.10548793, 0.89451207],
       [0.78730912, 0.21269088],
       [0.73374361, 0.26625639],
       [0.13501015, 0.86498985],
       [0.80128939, 0.19871061],
       [0.87356418, 0.12643582],
       [0.62766744, 0.37233256],
       [0.29441081, 0.70558919],
       [0.

In [18]:
clf_eval(y_test, pred, pred_proba)   ###질문

TypeError: 'numpy.ndarray' object is not callable

In [16]:
classifier.score(X_test, y_test)

0.7532467532467533

2.3 LogisticRegression에서 solver 파라미터의 역할과 어떤 것들이 있고 각각은 무엇을 의미하는지 작성하세요.

2.4 pred와 pred_proba는 각각 어떤 차이가 있는지 작성하세요.

pred는 모델로 예측된 y값들이고, pred_proba는 각 클래스 별 예측확률이다.

2.5 전체 데이터 중 약 65%가 Negative이므로 재현율 성능에 조첨을 맞추겠습니다. 정밀도 재현율 curve를 보고 임계값 별 정밀도 재현율 값의 변화를 확인하기 위해 이를 위한 함수를 작성하겠습니다.

In [ ]:
def precision_recall_curve_plot(y_test, pred_proba_c1):
    #threshold ndarray와 이 threshold에 따른 정밀도, 재현율 ndarray 추출.
    precisions, recalls, thresholds = precision_recall_curve(y_test, pred_proba_c1)

    #X축을 threshold값으로, Y축은 정밀도, 재현율 값으로 각각 Plot 수행, 정밀도는 점선으로 표시
    plt.figure(figsize=(8,6))
    threshold_boundary = thresholds.shape[0]
    plt.plot(thresholds, precisions[0:threshold_boundary], linestyle='--', label='precision')
    plt.plot(thresholds, recalls[0:threshold_boundary], label='recall')

    #threshold 값 X축의 Scale을 0.1단위로 변경
    start, end = plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1), 2))

    #X축, y축 label과 legend, grid 설정
    ### 직접 작성(xlabel - 'Threshold value', ylabel = 'Precision and Recall value'), legend, grid는 default값으로
    .......

2.6 작성한 함수를 바탕으로 임계값 별 정밀도와 재현율 값 변화를 확인하세요.

In [ ]:
pred_proba_c1 = lr_clf.predict_proba(X_test)[:,1]
precision_recall_curve_plot(y_test, pred_proba_c1)

2.7 그래프를 통해 확인했을 때 Threshold를 0.38정도로 설정했을 때 recall과 precision이 어느정도 균형이 맞음을 알 수 있습니다. 그러나 두 지표 모두 0.7 이하로 낮은 값이기에 다시 확인하도록 하겠습니다. 먼저 describe() 함수를 통해 개별 feature에 대한 분포도를 확인하세요.

2.8 위에서 확인해봤을 때 min 값이 0으로 되어 있는 feature가 다수 존재합니다. Glucose(포도당 수치), BMI 지수가 0인 것은 말이 안 되므로 이들을 평균값으로 대체하세요.

In [ ]:
### Glucose, BloodPressure, SkinThickness, Insulin, BMI에 대해 평균값으로 대체


2.9 이후 대체한 feature들에 대해 feature scaling을 진행하겠습니다. 모두 numeric feature이므로 X에 대해 StandardScaler를 일괄 적용하세요.

2.10 feature scaling은 왜 사용하고 어떤 것들이 있는지 작성하세요.

2.11 다시 에측하기에 앞서 위에서 scaling을 통해 변경된 X_scaled를 활용해 다시 train_test_split을 진행하세요(함수 파라미터 동일).

2.12 로지스틱 회귀로 앞서 진행한 것과 동일하게 진행하세요.

### 3. 자유 분석
성능을 높일 수 있는 방법들을 활용해 예측을 진행하세요.